In [26]:
import fitsio as fio
import numpy as np

In [27]:
clus = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog.fit')[-1] # Cluster catalog
shape = fio.FITS('/lsst/troxel/y1a1/lgt20_member_mcal_combined_mags_masked.fits')[-1] # Shape/member catalog
mem = fio.FITS('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog_members.fit')[-1] # Member catalog

In [28]:
cluster = clus.read()

In [29]:
cent_id = cluster['ID_CENT'][:,0] # Most probable central ID's
shape_id = shape['ID'].read() # Shape/member ID column
mem_id = mem['ID'].read() # Member ID column

In [30]:
# Mask shape/member catalog to select central galaxies
cent_mask = np.in1d(shape_id, cent_id, assume_unique=False)
cent = shape.read()[cent_mask]
r_mask = cent['R']==0
cent = cent[r_mask]

In [31]:
cent_sort = np.argsort(cent['MEM_MATCH_ID'])
cent = cent[cent_sort]

In [32]:
fio.write('/lsst/troxel/y1a1/lgt20_central_shapes_fmasked.fits', cent, clobber=True)

In [33]:
# Mask member catalog to select satellite galaxies
sat_mask = np.in1d(mem_id, cent['ID'], assume_unique=False, invert=True)
sat = mem.read()[sat_mask]
sat_cent_mask = np.in1d(sat['MEM_MATCH_ID'], cent['MEM_MATCH_ID'], assume_unique=False)
sat = sat[sat_cent_mask]

In [34]:
sat_sort = np.argsort(sat['MEM_MATCH_ID'])
sat = sat[sat_sort]

In [35]:
fio.write('/lsst/troxel/y1a1/lgt20_satellite_members_fmasked.fits', sat, clobber=True)

In [36]:
# Mask cluster catalog to select clusters with centrals that have shapes
clus_mask = np.in1d(cluster['MEM_MATCH_ID'], cent['MEM_MATCH_ID'], assume_unique=True)
clus_masked = cluster[clus_mask]

In [37]:
clus_sort = np.argsort(clus_masked['MEM_MATCH_ID'])
clus_masked = clus_masked[clus_sort]

In [38]:
fio.write('/lsst/troxel/y1a1/lgt20_cluster_fmasked.fits', clus_masked, clobber=True)

In [39]:
p_mem = sat['P']*sat['PFREE']
sat_p80_mask = p_mem >= 0.8
sat_p80 = sat[sat_p80_mask]

In [40]:
fio.write('/lsst/troxel/y1a1/lgt20_satellite_p80_fmasked.fits', sat_p80, clobber=True)

In [41]:
cent_p80_mask = np.in1d(cent['MEM_MATCH_ID'], sat_p80['MEM_MATCH_ID'], assume_unique=False)
cent_p80 = cent[cent_p80_mask]

In [42]:
fio.write('/lsst/troxel/y1a1/lgt20_central_p80_fmasked.fits', cent_p80, clobber=True)

In [43]:
sat_p20_mask = p_mem >= 0.2
sat_p20 = sat[sat_p20_mask]

In [44]:
fio.write('/lsst/troxel/y1a1/lgt20_satellite_p20_fmasked.fits', sat_p20, clobber=True)

In [45]:
cent_p20_mask = np.in1d(cent['MEM_MATCH_ID'], sat_p20['MEM_MATCH_ID'], assume_unique=False)
cent_p20 = cent[cent_p20_mask]

In [46]:
fio.write('/lsst/troxel/y1a1/lgt20_central_p20_fmasked.fits', cent_p20, clobber=True)

In [47]:
clus_sort = np.argsort(cluster['MEM_MATCH_ID'])
clus_sorted = cluster[clus_sort]

In [48]:
clus_p20_mask = np.in1d(clus_sorted['MEM_MATCH_ID'], cent_p20['MEM_MATCH_ID'], assume_unique=True)
clus_p20 = clus_sorted[clus_p20_mask]

In [49]:
fio.write('/lsst/troxel/y1a1/lgt20_cluster_p20_fmasked.fits', clus_p20, clobber=True)